In [1]:
%cd ../..

/home/aerdem/projects/nvidia/logits-processor-zoo


In [2]:
from examples.vllm.utils import vLLMRunner
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


example_prompts = [
"""
I am getting a lot of calls during the day. What is more important for me to consider when I buy a new phone?
a) Camera
b) Screen resolution
c) Operating System
d) Battery

""",
    
"""
Which user review doesn't belong to a summer dress?
a) Looks good
b) Keeps warm
c) Too long
d) Liked the color

"""
]

runner = vLLMRunner()

INFO 06-16 16:23:14 [__init__.py:243] Automatically detected platform cuda.
INFO 06-16 16:23:16 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-16 16:23:16 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-16 16:23:16 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
WARNING 06-16 16:23:17 [config.py:3135] Casting torch.bfloat16 to torch.float16.
INFO 06-16 16:23:22 [config.py:793] This model supports multiple tasks: {'generate', 'embed', 'reward', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 06-16 16:23:22 [cuda.py:87] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 06-16 16:23:22 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.0) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokeni

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-16 16:23:25 [default_loader.py:280] Loading weights took 0.59 seconds
INFO 06-16 16:23:25 [model_runner.py:1202] Model loading took 2.8876 GiB and 1.315691 seconds
INFO 06-16 16:23:27 [worker.py:291] Memory profiling takes 1.57 seconds
INFO 06-16 16:23:27 [worker.py:291] the current vLLM instance can use total_gpu_memory (23.56GiB) x gpu_memory_utilization (0.90) = 21.20GiB
INFO 06-16 16:23:27 [worker.py:291] model weights take 2.89GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 16.22GiB.
INFO 06-16 16:23:27 [executor_base.py:112] # cuda blocks: 37953, # CPU blocks: 9362
INFO 06-16 16:23:27 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 18.53x
INFO 06-16 16:23:28 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 3.11 seconds


In [3]:
mclp = MultipleChoiceLogitsProcessor(runner.tokenizer, choices=["a", "b", "c", "d"], delimiter=")")

example_prompts = example_prompts * 32
len(example_prompts)

64

In [4]:
%%time

runner.generate_response(example_prompts, [mclp], max_tokens=1)

Prompt: 
I am getting a lot of calls during the day. What is more important for me to consider when I buy a new phone?
a) Camera
b) Screen resolution
c) Operating System
d) Battery


a
-----END-----

Prompt: 
Which user review doesn't belong to a summer dress?
a) Looks good
b) Keeps warm
c) Too long
d) Liked the color


b
-----END-----

Prompt: 
I am getting a lot of calls during the day. What is more important for me to consider when I buy a new phone?
a) Camera
b) Screen resolution
c) Operating System
d) Battery


a
-----END-----

Prompt: 
Which user review doesn't belong to a summer dress?
a) Looks good
b) Keeps warm
c) Too long
d) Liked the color


b
-----END-----

Prompt: 
I am getting a lot of calls during the day. What is more important for me to consider when I buy a new phone?
a) Camera
b) Screen resolution
c) Operating System
d) Battery


a
-----END-----

Prompt: 
Which user review doesn't belong to a summer dress?
a) Looks good
b) Keeps warm
c) Too long
d) Liked the color




In [5]:
from logits_processor_zoo.vllm import ForceLastPhraseLogitsProcessor

example_prompts = [
    """
    Retrieved information from: https://en.wikipedia.org/wiki/Bulbasaur
    Bulbasaur is a fictional Pokémon species in Nintendo and Game Freak's Pokémon franchise. 
    Designed by Atsuko Nishida, Bulbasaur is a Grass and Poison-type, first appearing in Pocket Monsters: Red and Green (Pokémon Red and Blue outside Japan) as a starter Pokémon. 
    Since then, it has reappeared in sequels, spin-off games, related merchandise, and animated and printed adaptations of the franchise. 
    It is a central character in the Pokémon anime, being one of Ash Ketchum's main Pokémon for the first season, with a different one later being obtained by supporting character May. 
    It is featured in various manga and is owned by protagonist Red in Pokémon Adventures.
    
    What is Bulbasaur?
    """,
]

reference = ForceLastPhraseLogitsProcessor("\n\nReferences:", runner.tokenizer)

example_prompts = example_prompts * 64
len(example_prompts)

64

In [6]:
%%time

runner.generate_response(example_prompts, [reference])

Prompt: 
    Retrieved information from: https://en.wikipedia.org/wiki/Bulbasaur
    Bulbasaur is a fictional Pokémon species in Nintendo and Game Freak's Pokémon franchise. 
    Designed by Atsuko Nishida, Bulbasaur is a Grass and Poison-type, first appearing in Pocket Monsters: Red and Green (Pokémon Red and Blue outside Japan) as a starter Pokémon. 
    Since then, it has reappeared in sequels, spin-off games, related merchandise, and animated and printed adaptations of the franchise. 
    It is a central character in the Pokémon anime, being one of Ash Ketchum's main Pokémon for the first season, with a different one later being obtained by supporting character May. 
    It is featured in various manga and is owned by protagonist Red in Pokémon Adventures.
    
    What is Bulbasaur?
    
Bulbasaur is a fictional Pokémon species in the Pokémon franchise. It is a Grass and Poison-type Pokémon, designed by Atsuko Nishida. Bulbasaur first appeared in Pocket Monsters: Red and Green (Po